# db_queries
#### Notebook for running python sqlite3 commands for querying database

In [1]:
import matplotlib.pyplot as plt
import numpy as np

ImportError: No module named matplotlib.pyplot

In [2]:
import sqlite3

In [3]:
# path on rte
db_path = "/home/rte/data/db/arxiv_db_images.sqlite3"

In [ ]:
# path on kt
db_path = "/home/kt/rte/db/arxiv_db_images.sqlite3"

In [4]:
# Here we import the sqlite3 database and create a cursor

db = sqlite3.connect(db_path)
c = db.cursor()

Get the pragma table info for each table

In [5]:
c.execute('PRAGMA TABLE_INFO({})'.format("metadata"))
info = c.fetchall()

print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
for col in info:
    print(col)


Column Info:
ID, Name, Type, NotNull, DefaultVal, PrimaryKey
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'identifier', 'TEXT', 0, None, 0)
(2, 'created', 'TEXT', 0, None, 0)
(3, 'cat', 'TEXT', 0, None, 0)
(4, 'authors', 'TEXT', 0, None, 0)
(5, 'title', 'TEXT', 0, None, 0)
(6, 'abstract', 'TEXT', 0, None, 0)
(7, 'licence', 'TEXT', 0, None, 0)


In [6]:
c.execute('PRAGMA TABLE_INFO({})'.format("images"))
info = c.fetchall()

print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
for col in info:
    print(col)


Column Info:
ID, Name, Type, NotNull, DefaultVal, PrimaryKey
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'identifier', 'TEXT', 0, None, 0)
(2, 'filename', 'TEXT', 0, None, 0)
(3, 'filesize', 'INT', 0, None, 0)
(4, 'path', 'TEXT', 0, None, 0)
(5, 'x', 'INT', 0, None, 0)
(6, 'y', 'INT', 0, None, 0)
(7, 'imageformat', 'TEXT', 0, None, 0)


Quick test to retrive one row

In [7]:
c.execute('''
    SELECT * 
    FROM images 
    ORDER BY ROWID ASC 
    LIMIT 1
''')
rows = c.fetchall()
for row in rows:
    print(row)

(1, 'cond-mat0606521', 'lay_f2.eps', 29311, './0606/cond-mat0606521', 250, 180, 'PS')


In [8]:
c.execute('''
    SELECT * 
    FROM metadata 
    ORDER BY ROWID ASC 
    LIMIT 1
''')
rows = c.fetchall()
for row in rows:
    print(row)

(1, '0704.0026', '2007-03-31', 'math.RA', "['de Marrais, Robert P. C.; ']", 'Placeholder Substructures II: Meta-Fractals, Made of Box-Kites, Fill\n  Infinite-Dimensional Skies', '  Zero-divisors (ZDs) derived by Cayley-Dickson Process (CDP) from\nN-dimensional hypercomplex numbers (N a power of 2, at least 4) can represent\nsingularities and, as N approaches infinite, fractals -- and thereby,scale-free\nnetworks. Any integer greater than 8 and not a power of 2 generates a\nmeta-fractal or "Sky" when it is interpreted as the "strut constant" (S) of an\nensemble of octahedral vertex figures called "Box-Kites" (the fundamental\nbuilding blocks of ZDs). Remarkably simple bit-manipulation rules or "recipes"\nprovide tools for transforming one fractal genus into others within the context\nof Wolfram\'s Class 4 complexity.\n', '')


Find any duplicate identifiers

In [9]:
# look for duplicate rows in the metadata table
c.execute('''
    SELECT identifier, COUNT(identifier)
    FROM metadata
    GROUP BY identifier
    HAVING COUNT(identifier) > 1
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# look for duplicate rows in the images table (brings up totals for images by article ID)
c.execute('''
    SELECT identifier, COUNT(identifier)
    FROM images
    GROUP BY identifier
    HAVING COUNT(identifier) > 0
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [11]:
# Find the earliest date of an article

c.execute('''
    SELECT created, identifier
    FROM metadata
    WHERE created IS NOT NULL
    ORDER BY created ASC
    LIMIT 20
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

('1986-04-25', 'physics9403001')
('1988-11-11', 'hep-th9108028')
('1989-04-14', 'math9201239')
('1989-10-26', 'math9201205')
('1989-10-26', 'math9201203')
('1989-10-26', 'math9201204')
('1989-11-09', 'math9201206')
('1989-11-17', 'math9201207')
('1989-12-31', 'math9201303')
('1989-12-31', 'cs9301111')
('1990-01-14', 'math9201241')
('1990-01-14', 'math9201240')
('1990-01-14', 'math9201242')
('1990-01-17', 'math9201271')
('1990-02-15', 'math9201208')
('1990-02-18', 'math9201209')
('1990-03-15', 'math9201210')
('1990-03-27', 'math9201211')
('1990-03-31', 'cs9301112')
('1990-04-19', 'math9201272')


In [ ]:
# Find the earliest dated articles and get the identifiers

c.execute('''
    SELECT images.identifier, metadata.created, metadata.identifier 
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier
    WHERE metadata.created IS NOT NULL
    ORDER BY created ASC
    LIMIT 20
    ''')
    
rows = c.fetchall()
for row in rows:
    print(row)

In [12]:
# Find total number of rows in metadata

c.execute('''
    SELECT count(*)
    FROM metadata
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

(1506562,)


In [13]:
# Find total number of rows in images

c.execute('''
    SELECT count(*)
    FROM images
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

(10061222,)


In [ ]:
# Get average of the x size column

c.execute('''
    SELECT avg(x)
    FROM images
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get average of the y size column

c.execute('''
    SELECT avg(y)
    FROM images
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find where there are NULL values for x

c.execute('''
    SELECT id, identifier, path, filename, filesize, x, y, imageformat
    FROM images
    WHERE x is null or x = ''
    AND y is null or y = ''
    AND imageformat is null or imageformat = ''
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find where there are NULL values for y

c.execute('''
    SELECT id, identifier, path, filename, filesize, x, y, imageformat
    FROM images
    WHERE y is null or y = ''
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
print(len(rows))

In [ ]:
# Find where there are NULL values for identifier

c.execute('''
    SELECT id, identifier, path, filesize, x, y, imageformat
    FROM images
    WHERE identifier is null or identifier = ''
    ''')
rows = c.fetchall()
for row in rows:
    print(row)
    
# no result is a good result!

In [ ]:
# Find where there are NULL values for filesize

c.execute('''
    SELECT id, identifier, path, filesize, x, y, imageformat
    FROM images
    WHERE filesize is null or filesize = ''
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
print(len(rows))

In [ ]:
# save (commit) changes

db.commit()

In [ ]:
# Get total number of images per article

c.execute('''
    SELECT images.identifier, metadata.cat, count(images.identifier)    
    FROM images 
    LEFT JOIN metadata ON images.identifier = metadata.identifier 
    GROUP BY images.identifier
    ORDER BY count(images.identifier)
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get total number of images per category

c.execute('''
    SELECT metadata.cat, count(images.identifier)
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier 
    GROUP BY metadata.cat
    ORDER BY count(images.identifier) DESC    
    LIMIT 200
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get total number of images per primary category only

c.execute('''
    SELECT substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1), count(images.identifier)
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier 
    GROUP BY substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1)
    ORDER BY count(images.identifier) DESC    
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
fname = "stats_images_by_category.txt"
f = open(fname, "w+")
for row in rows:
    f.write(row[0] + ", " + str(row[1]) + "\n")
f.close()

In [ ]:
print(len(rows))

In [ ]:
# Get total number of images per primary category only - with a specific filter

c.execute('''
    SELECT substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1), count(images.identifier)
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier
    WHERE substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1) = "cond-mat.mes-hall"
    GROUP BY substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1)
    ORDER BY count(images.identifier) DESC    
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get number of articles with no category
# this seems to give no results

c.execute('''
    SELECT identifier, cat
    FROM metadata
    WHERE cat IS NULL OR cat = '' OR cat = 'None'
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find where there is no matching metadata for a given image

c.execute('''
    SELECT count(images.identifier), images.identifier
    FROM images
    LEFT JOIN metadata ON metadata.identifier = images.identifier
    WHERE metadata.identifier IS NULL
    GROUP BY images.identifier
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
print(len(rows))

In [ ]:
print(sum(row[0] for row in rows))

In [ ]:
# Find where there is no identifier for an image

c.execute('''
    SELECT identifier, id, filename, filesize, path, x, y
    FROM images
    WHERE identifier IS NULL OR identifier = ''
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# More looking for specific entries with no identifier

c.execute('''
    SELECT identifier, id
    FROM images
    WHERE identifier IS NULL OR identifier = ''
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find number of images in each month/year

c.execute('''
    SELECT count(images.identifier), strftime("%m-%Y", metadata.created) as 'mY'
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier 
    GROUP BY strftime("%m-%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [25]:
# Find number of images by year

c.execute('''
    SELECT count(images.identifier), strftime("%Y", metadata.created) as 'Y'
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier 
    GROUP BY strftime("%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

(11, '1988')
(130, '1990')
(146, '1991')
(972, '1992')
(6885, '1993')
(20136, '1994')
(26763, '1995')
(51088, '1996')
(75713, '1997')
(101662, '1998')
(122189, '1999')
(137860, '2000')
(150775, '2001')
(171813, '2002')
(190805, '2003')
(220669, '2004')
(243900, '2005')
(266790, '2006')
(303204, '2007')
(342292, '2008')
(388500, '2009')
(431434, '2010')
(497368, '2011')
(560836, '2012')
(659050, '2013')
(729214, '2014')
(858174, '2015')
(1003842, '2016')
(1130770, '2017')
(1368231, '2018')


In [27]:
total = 0
for row in rows:
    total += row[0]
print(total)

10061222


In [ ]:
# Find number of articles in each month/year

c.execute('''
    SELECT count(metadata.identifier), strftime("%m-%Y", metadata.created) as 'mY'
    FROM metadata
    GROUP BY strftime("%m-%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [23]:
# Find number of articles by year

c.execute('''
    SELECT count(metadata.identifier), strftime("%Y", metadata.created) as 'Y'
    FROM metadata
    GROUP BY strftime("%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

(1, '1986')
(1, '1988')
(8, '1989')
(25, '1990')
(370, '1991')
(3181, '1992')
(6728, '1993')
(10085, '1994')
(12994, '1995')
(15876, '1996')
(19621, '1997')
(24174, '1998')
(27694, '1999')
(30672, '2000')
(33127, '2001')
(36102, '2002')
(39389, '2003')
(43719, '2004')
(46863, '2005')
(50303, '2006')
(55768, '2007')
(58796, '2008')
(64077, '2009')
(70283, '2010')
(76604, '2011')
(84385, '2012')
(92864, '2013')
(97593, '2014')
(105124, '2015')
(113422, '2016')
(123750, '2017')
(140242, '2018')
(22721, '2019')


In [24]:
total = 0
for row in rows:
    total += row[0]
print(total)

1506562


In [31]:
# Find number of articles by year by category

c.execute('''
    SELECT count(metadata.identifier), strftime("%Y", metadata.created) as 'Y'
    FROM metadata
    WHERE substr(trim(cat),1,instr(trim(cat)||' ',' ')-1) = 'cs.CV'
    GROUP BY strftime("%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

(2, '1998')
(1, '1999')
(10, '2000')
(4, '2001')
(4, '2002')
(16, '2003')
(17, '2004')
(22, '2005')
(25, '2006')
(38, '2007')
(60, '2008')
(100, '2009')
(242, '2010')
(277, '2011')
(422, '2012')
(662, '2013')
(1096, '2014')
(1859, '2015')
(3086, '2016')
(4912, '2017')
(7262, '2018')
(1086, '2019')


In [ ]:
# Find number of articles by year by category

c.execute('''
    SELECT count(metadata.identifier), strftime("%Y", metadata.created) as 'Y'
    FROM metadata
    WHERE substr(trim(cat),1,instr(trim(cat)||' ',' ')-1) = 'cs.CV'
    AND strftime("%Y", metadata.created) = '2018'
    GROUP BY strftime("%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
print(len(rows))

In [ ]:
# values = np.array()
values = [item[0] for item in rows]
plt.plot(values)

In [ ]:
# Find number of images per each different imageformat

c.execute('''
    SELECT imageformat, count(imageformat)
    FROM images
    GROUP BY imageformat
    ORDER BY count(imageformat) DESC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get only the first listed category

c.execute('''
    SELECT identifier, cat, substr(trim(cat),1,instr(trim(cat)||' ',' ')-1)
    FROM metadata
    LIMIT 20
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get the licence information for number of articles

c.execute('''
    SELECT licence, COUNT(licence)
    FROM metadata
    GROUP BY licence
    HAVING COUNT(identifier) > 0
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
c.execute('''
    SELECT COUNT(*)
    FROM images
    ''')
rows = c.fetchall()
for row in rows:
    print(row)